In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt


import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/')
from spicy_snow.download.snowex_lidar import download_dem

import pickle
with open('/Users/zachkeskinen/Documents/spicy-snow/SnowEx-Data/Cameron_test.pkl', 'rb') as f:
    ds = pickle.load(f)

In [ ]:
for date in ds.time:
    pass

In [ ]:
f'{"Cameron"}_{(date).dt.strftime("%y-%m-%d").values}'

In [ ]:
def make_site_ds(site):
    ds = xr.Dataset()

    for img_type in ['SD', 'VH', 'DEM']:
        files = glob(join(lidar_dir, f'*_{img_type}_*_{site}_*.tif'))
        assert files, f"No files found for {img_type} at {site_name}"
        
        imgs = []
        for f in files:
            print(f)
            date = pd.to_datetime(basename(f).split('_')[-2])

            img = rxa.open_rasterio(f, masked = True)

            img = img.squeeze(dim = 'band')
            
            img = img.expand_dims(time = [date])

            imgs.append(img)
    
        ds['lidar-' + img_type.lower()] = xr.concat(imgs, dim = 'time')

    ds = ds.rio.reproject('EPSG:4326')
    
    return ds

In [ ]:
ds

In [ ]:
from glob import glob
from os.path import join, basename
import os
lidar_dir = '/Users/zachkeskinen/Documents/spicy-snow/data/lidar'

sites = {'USCOCP': 'Cameron', 'USCOFR': 'Frasier', 'USIDBS': 'Banner', 'USIDDC': 'Dry_Creek',
         'USIDMC': 'Mores', 'USUTLC': 'Little_Cottonwood'}

sites = {'USIDDC': 'Dry_Creek'}
# sites = {'USIDBS': 'Banner'}

for site, site_name in sites.items():

    lidar_ds_dc = make_site_ds(site)

In [ ]:
for date in ds.time:
    if date.dt.month < 8:
        last_year = date.dt.year - 1
        date1 = pd.to_datetime(f'{int(date.dt.year - 1)}-08-01')
    else:
        date1 = pd.to_datetime(f'{int(date.dt.year)}-08-01')
    print((date1, pd.to_datetime(date.values)))
    date1 = date - pd.Timedelta('26 days')

    # spicy_ds = retrieve_snow_depth(area = area, dates = (date1, pd.to_datetime(date.values)), work_dir = '../data/', job_name = f'spicy-{site}')
    import pickle
    with open('/Users/zachkeskinen/Documents/spicy-snow/data/banner_lidar.pkl', 'rb') as f:
        spicy_ds = pickle.load(f)
    
    lidar_ds.rio.reproject_match(spicy_ds)

    # ds = xr.merge([spicy_ds, lidar_ds], combine_attrs = 'drop_conflicts')

In [ ]:
lidar_ds

In [ ]:
lidar_ds['sd'].rio.write_nodata(np.nan)

In [ ]:
lidar_ds_re = lidar_ds.rio.reproject_match(spicy_ds)

In [ ]:
ds = xr.merge([spicy_ds, nanlidards], combine_attrs = 'drop_conflicts')

In [ ]:
nanlidards = lidar_ds_re.where(lidar_ds_re < 1000).where(lidar_ds_re > -1000)

In [ ]:
nanlidards['sd'].rio.nodata

In [ ]:
ds[['sd', 'vh', 'dem', 'snow_depth', 's1']]